# NOTE: This is not a cookbook

This is a testing notebook in order to make sure that multimodal works.

Cookbook is on the way, but if you have particular ideas, message @isaac on discord

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import dspy
from dspy.datasets import DataLoader
from dspy.evaluate.metrics import answer_exact_match
from typing import List
from dspy.evaluate import Evaluate

import dotenv
import litellm

litellm.suppress_debug_info = True

dotenv.load_dotenv()

def debug_exact_match(example, pred, trace=None, frac=1.0):
    print(example.inputs())
    print(example.answer)
    print(pred)
    # print(trace)
    # print(frac)
    return answer_exact_match(example, pred, trace, frac)

In [3]:
# vllm serve Qwen/Qwen2-VL-7B-Instruct --trust-remote-code --limit-mm-per-prompt image=16 --seed 42 --pipeline-parallel-size 2
qwen_lm = dspy.LM(model="openai/Qwen/Qwen2-VL-7B-Instruct", api_base="http://localhost:8000/v1", api_key="sk-fake-key", max_tokens=5000)
haiku_lm = dspy.LM(model="anthropic/claude-3-haiku-20240307", max_tokens=4096)
# vllm serve meta-llama/Llama-3.2-11B-Vision-Instruct --trust-remote-code --limit-mm-per-prompt image=16 --seed 42 --enforce-eager --max-num-seqs 48
llama_lm = dspy.LM(model="openai/meta-llama/Llama-3.2-11B-Vision-Instruct", api_base="http://localhost:8000/v1", api_key="sk-fake-key", max_tokens=5000)
internlm_lm = dspy.LM(model="openai/OpenGVLab/InternVL2-8B", api_base="http://localhost:8000/v1", api_key="sk-fake-key", max_tokens=5000)
gpt_lm = dspy.LM(model="openai/gpt-4o-mini", max_tokens=5000)
all_lms = [qwen_lm, haiku_lm, llama_lm, gpt_lm]

dspy.settings.configure(lm=gpt_lm)


In [ ]:
class DogPictureSignature(dspy.Signature):
    """Answer the question based on the image."""
    image: dspy.Image = dspy.InputField()
    question: str = dspy.InputField()
    answer: str = dspy.OutputField()

class DogPicture(dspy.Module):
    def __init__(self) -> None:
        self.predictor = dspy.ChainOfThought(DogPictureSignature)
    
    def __call__(self, **kwargs):
        return self.predictor(**kwargs)

dog_picture = DogPicture()

example = dspy.Example(image=dspy.Image.from_url("https://i.pinimg.com/564x/78/f9/6d/78f96d0314d39a1b8a849005123e166d.jpg"), question="What is the breed of the dog in the image?").with_inputs("image", "question")
print(dog_picture(**example.inputs()))

In [ ]:
# qwen_lm.inspect_history()
import rich
rich.print(qwen_lm.history)


In [ ]:
p = dspy.Predict("question -> answer")
p(question="What is the capital of France?")
qwen_lm.inspect_history()

In [4]:
%%capture
from concurrent.futures import ThreadPoolExecutor

input_keys = tuple([f"image_{i}" for i in range(1, 3)] + ["question", "options"])
subsets = ['Accounting', 'Agriculture', 'Architecture_and_Engineering', 'Art', 'Art_Theory', 'Basic_Medical_Science', 'Biology', 'Chemistry', 'Clinical_Medicine', 'Computer_Science', 'Design', 'Diagnostics_and_Laboratory_Medicine', 'Economics', 'Electronics', 'Energy_and_Power', 'Finance', 'Geography', 'History', 'Literature', 'Manage', 'Marketing', 'Materials', 'Math', 'Mechanical_Engineering', 'Music', 'Pharmacy', 'Physics', 'Psychology', 'Public_Health', 'Sociology']

devset = []
valset = []
with ThreadPoolExecutor(max_workers=len(subsets)) as executor:
    def load_dataset(subset_index_subset):
        subset_index, subset = subset_index_subset
        dataset = DataLoader().from_huggingface("MMMU/MMMU", subset, split=["dev", "validation"], input_keys=input_keys)
        return subset_index, dataset["dev"], dataset["validation"]
    
    results = list(executor.map(load_dataset, enumerate(subsets)))
    
    results.sort(key=lambda x: x[0])
    
    for _, dev, val in results:
        devset.extend(dev)
        valset.extend(val)

print(len(devset))
print(len(valset))

In [5]:
import ast

def count_images(dataset):
    image_counts = {i: 0 for i in range(6)}  # Initialize counts for 0 to 2 images
    for example in dataset:
        count = sum(1 for key in example.inputs().keys() if key.startswith('image_') and example.inputs()[key] is not None)
        image_counts[count] += 1
    return image_counts

def count_multiple_choice_questions(dataset):
    return sum(1 for example in dataset if example["question_type"] == "multiple-choice")
max_images = 5

num_images = 1

devset_filtered = [example for example in devset if sum(1 for key in example.inputs().keys() if key.startswith('image_') and example.inputs()[key] is not None) == num_images]
valset_filtered = [example for example in valset if sum(1 for key in example.inputs().keys() if key.startswith('image_') and example.inputs()[key] is not None) == num_images]

devset_filtered = [example for example in devset_filtered if example["question_type"] == "multiple-choice"]
valset_filtered = [example for example in valset_filtered if example["question_type"] == "multiple-choice"]

def update_example_image_key(example):
    example_copy = example.copy()
    example_copy["image"] = dspy.Image.from_PIL(example_copy["image_1"])
    return example_copy.with_inputs(*example.inputs().keys(), "image")



devset_filtered = list(map(update_example_image_key, devset_filtered))
valset_filtered = list(map(update_example_image_key, valset_filtered))

devset_image_counts = count_images(devset_filtered)
valset_image_counts = count_images(valset_filtered)

devset_multiple_choice_questions = count_multiple_choice_questions(devset_filtered)
valset_multiple_choice_questions = count_multiple_choice_questions(valset_filtered)

print("Image counts in devset:")
for count, num_examples in devset_image_counts.items():
    print(f"{count} image(s): {num_examples} examples")

print("\nImage counts in valset:")
for count, num_examples in valset_image_counts.items():
    print(f"{count} image(s): {num_examples} examples")

print("\nMultiple choice questions in devset:")
print(devset_multiple_choice_questions, "out of", len(devset_filtered))
print("\nMultiple choice questions in valset:")
print(valset_multiple_choice_questions, "out of", len(valset_filtered))

def convert_multiple_choice_to_letter(dataset):
    new_dataset = []
    for example in dataset:
        if example["question_type"] == "multiple-choice":
            # print(example["options"])
            options = ast.literal_eval(example["options"])
            example["choices"] = str([chr(65 + i) + ". " + option for i, option in enumerate(options)])
        else:
            example["choices"] = str(ast.literal_eval(example["options"]))
            if example["choices"] == []:
                example["choices"] = "Free response"

        updated_example = example.with_inputs(*example.inputs().keys(), "choices")
        new_dataset.append(updated_example)
    return new_dataset

print(devset_filtered[0])
updated_devset = convert_multiple_choice_to_letter(devset_filtered)
print(updated_devset[0])
updated_valset = convert_multiple_choice_to_letter(valset_filtered)




Image counts in devset:
0 image(s): 0 examples
1 image(s): 137 examples
2 image(s): 0 examples
3 image(s): 0 examples
4 image(s): 0 examples
5 image(s): 0 examples

Image counts in valset:
0 image(s): 0 examples
1 image(s): 805 examples
2 image(s): 0 examples
3 image(s): 0 examples
4 image(s): 0 examples
5 image(s): 0 examples

Multiple choice questions in devset:
137 out of 137

Multiple choice questions in valset:
805 out of 805
Example({'id': 'dev_Accounting_1', 'question': 'Each of the following situations relates to a different company. <image 1> For company B, find the missing amounts.', 'options': "['$63,020', '$58,410', '$71,320', '$77,490']", 'explanation': '', 'image_1': <PIL.PngImagePlugin.PngImageFile image mode=RGBA size=1234x289 at 0x7921F0091E10>, 'image_2': None, 'image_3': None, 'image_4': None, 'image_5': None, 'image_6': None, 'image_7': None, 'img_type': "['Tables']", 'answer': 'D', 'topic_difficulty': 'Easy', 'question_type': 'multiple-choice', 'subfield': 'Financi

In [26]:
import re
from typing import Literal
class MMMUSignature(dspy.Signature):
    """Answer with the letter of the correct answer."""

    question: str = dspy.InputField()
    image: dspy.Image = dspy.InputField()
    choices: List[str] = dspy.InputField()
    answer: Literal["A", "B", "C", "D", "E"] = dspy.OutputField()

class MMMUModule(dspy.Module):
    def __init__(self, cot=True):
        super().__init__()
        if cot:
            self.predictor = dspy.ChainOfThought(MMMUSignature)
        else:
            self.predictor = dspy.Predict(MMMUSignature)

    def __call__(self, **kwargs):
        # Clean up predictions
        prediction = self.predictor(**kwargs)
        # Multiple choice case
        if "A." in kwargs["choices"]:
            # regex to extract A, B, C, or D, or E
            answer = re.search(r'[A-E]', prediction["answer"])
            if not answer:
                answer = prediction["answer"]
            else:
                answer = answer.group(0)
            prediction["answer"] = answer
        # Free response case
        return prediction


In [28]:


sample_input = updated_devset[0]
# print(sample_input.inputs())
# print(encode_image(sample_input.inputs()["image_1"]))
mmmu = MMMUModule()
print(sample_input.inputs())
print(mmmu(**sample_input.inputs()))
print(sample_input.answer)

evaluate_mmmu = Evaluate(metric=answer_exact_match, num_threads=50, devset=updated_valset, display_progress=True, max_errors=500, return_outputs=True)
gpt_lm.inspect_history()

Example({'question': 'Each of the following situations relates to a different company. <image 1> For company B, find the missing amounts.', 'options': "['$63,020', '$58,410', '$71,320', '$77,490']", 'image_1': <PIL.PngImagePlugin.PngImageFile image mode=RGBA size=1234x289 at 0x7921F0091E10>, 'image_2': None, 'image': <PIL.PngImagePlugin.PngImageFile image mode=RGBA size=1234x289 at 0x7921F0091E10>, 'choices': "['A. $63,020', 'B. $58,410', 'C. $71,320', 'D. $77,490']"}) (input_keys={'image_2', 'image', 'question', 'image_1', 'choices', 'options'})
field info:  annotation=str required=True json_schema_extra={'__dspy_field_type': 'input', 'prefix': 'Question:', 'desc': '${question}'} value:  {question} type:  <class 'str'>
field info:  annotation=Image required=True json_schema_extra={'__dspy_field_type': 'input', 'prefix': 'Image:', 'desc': '${image}'} value:  {image} type:  <class 'str'>
field info:  annotation=List[str] required=True json_schema_extra={'__dspy_field_type': 'input', 'pr

In [8]:

def test_lm(lm, cot=False):
    if lm.model == "openai/gpt-4o-mini":
        num_threads = 10
    else:
        num_threads = 30
    evaluate_mmmu = Evaluate(metric=answer_exact_match, num_threads=num_threads, devset=updated_valset, display_progress=True, max_errors=500, return_outputs=True)
    mmmu = MMMUModule(cot=cot)
    with dspy.context(lm=lm):
        scores, outputs = evaluate_mmmu(mmmu)
        num_bad_format = sum(1 for example in outputs if example[1].get("answer", None) is None)
        return scores, num_bad_format

# res1 = test_lm(qwen_lm)
# res1_cot = test_lm(qwen_lm, cot=True)
# test_lm(haiku_lm)
# test_lm(llama_lm)
# res1 = test_lm(internlm_lm)
res2 = test_lm(gpt_lm)
res2_cot = test_lm(gpt_lm, cot=True)
# Results:
# MMMU Val(single image only, multiple choice only), N=805
# Temp 0, max_tokens=5k

# 4o-mini:
# Reported: 59.4

# Measured (cot, predict): 60.0, 56.4
# Num bad format (cot, predict): 0, 1

# qwen-7b
# Reported: 54.1
# Measured (cot, predict): 49.0, 49.69
# Num bad format (cot, predict): 17, 0
print("MMMU Validation Set (single image only, multiple choice only), N=805")
print("Temp 0, max_tokens=5k")
# print("qwen-7b")
# print("Reported:", 54.1)
# print("Measured (cot, predict):", f"{res1_cot[0]:.1f}, {res1[0]:.2f}")
# print("Num bad format (cot, predict):", f"{res1_cot[1]}, {res1[1]}")
# print()
print("gpt-4o-mini")
print("Reported:", 59.4)
print("Measured (cot, predict):", f"{res2_cot[0]:.1f}, {res2[0]:.2f}") 
print("Num bad format (cot, predict):", f"{res2_cot[1]}, {res2[1]}")


Average Metric: 366.0 / 805  (45.5): 100%|██████████| 805/805 [01:02<00:00, 12.81it/s]
2024/11/13 00:29:32 INFO dspy.evaluate.evaluate: Average Metric: 366.0 / 805 (45.5%)
Average Metric: 356.0 / 805  (44.2): 100%|██████████| 805/805 [04:18<00:00,  3.12it/s]
2024/11/13 00:33:50 INFO dspy.evaluate.evaluate: Average Metric: 356.0 / 805 (44.2%)


MMMU Validation Set (single image only, multiple choice only), N=805
Temp 0, max_tokens=5k
qwen-7b
Reported: 54.1
gpt-4o-mini
Reported: 59.4
Measured (cot, predict): 44.2, 45.47
Num bad format (cot, predict): 51, 1


In [9]:
gpt_lm.inspect_history()





[2024-11-13T00:33:50.703870]

System message:

Your input fields are:
1. `question` (str)
2. `image` (Image)
3. `choices` (list[str])

Your output fields are:
1. `reasoning` (str)
2. `answer` (Literal[A, B, C, D, E])

All interactions will be structured in the following way, with the appropriate values filled in.

[[ ## question ## ]]
{question}

[[ ## image ## ]]
{image}

[[ ## choices ## ]]
{choices}

[[ ## reasoning ## ]]
{reasoning}

[[ ## answer ## ]]
{answer}        # note: the value you produce must be one of: A; B; C; D; E

[[ ## completed ## ]]

In adhering to this structure, your objective is: 
        Answer with the letter of the correct answer.


User message:

[[ ## question ## ]]
Some controversial groups that may be mislabelled as <image 1> include:

[[ ## image ## ]]
<PIL.PngImagePlugin.PngImageFile image mode=RGBA size=1080x746 at 0x7921E40DD150>

[[ ## choices ## ]]
['A. Scientology and the Hare Krishna', "B. the Peoples Temple and Heven's Gate", 'C. the Branch D

In [ ]:
scores, outputs = evaluate_mmmu(mmmu)
# lm.inspect_history()


In [ ]:
from collections import Counter
c = Counter([outputs[i][1].get("answer", "nothing returned") for i in range(len(outputs))])
non_letters = sum([1 for output in outputs if output[1].get("answer", "nothing returned") not in ["A", "B", "C", "D"]])
print(c)
print(non_letters)




In [ ]:
mc_correct = sum(outputs[i][2] for i in range(len(outputs)) if outputs[i][0]["question_type"] == "multiple-choice")
total_mc = sum(1 for example in outputs if example[0]["question_type"] == "multiple-choice")
print(mc_correct, total_mc)
print(mc_correct / total_mc)
print(sum(outputs[i][1].get("answer", None) is None for i in range(len(outputs))))

# Note: Run above here

# Make sure that multiple images work

## No examples

In [ ]:
import PIL
def set_image_to_black_square(example, key):
    example_copy = example.copy()
    example_copy[key] = PIL.Image.open("black_image_300x300.png")
    return example_copy.with_inputs(*example.inputs().keys())

print(updated_devset[0]["image_1"])
print(updated_devset[0]["image_2"])
examples_no_image_1 = list(map(lambda x: set_image_to_black_square(x, "image_1"), updated_valset))
print(examples_no_image_1[0]["image_1"] == PIL.Image.open("black_image_300x300.png"))
print(examples_no_image_1[0]["image_2"] == PIL.Image.open("black_image_300x300.png"))
examples_no_image_2 = list(map(lambda x: set_image_to_black_square(x, "image_2"), updated_valset))
print(examples_no_image_2[0]["image_1"] == PIL.Image.open("black_image_300x300.png"))
print(examples_no_image_2[0]["image_2"] == PIL.Image.open("black_image_300x300.png"))

examples_no_actual_image = list(map(lambda x: set_image_to_black_square(x, "image_1"), updated_valset))
examples_no_actual_image = list(map(lambda x: set_image_to_black_square(x, "image_2"), examples_no_actual_image))
print(examples_no_actual_image[0]["image_1"] == PIL.Image.open("black_image_300x300.png"))
print(examples_no_actual_image[0]["image_2"] == PIL.Image.open("black_image_300x300.png"))


In [ ]:
mmmu = MMMUModule()
print(examples_no_image_1[0].inputs())
print(mmmu(**examples_no_image_1[0].inputs()))

print(examples_no_image_2[0].inputs())
print(mmmu(**examples_no_image_2[0].inputs()))


In [ ]:
normal = evaluate_mmmu(mmmu, devset=updated_valset)
no_image_1 = evaluate_mmmu(mmmu, devset=examples_no_image_1)
no_image_2 = evaluate_mmmu(mmmu, devset=examples_no_image_2)
no_actual_image = evaluate_mmmu(mmmu, devset=examples_no_actual_image)
print("Testing on MMMU validation set (N=", len(updated_valset), ")")
print("Score with both images:", normal)
print("Score with image_1 set to black square:", no_image_1)
print("Score with image_2 set to black square:", no_image_2)
print("Score with both images set to black squares:", no_actual_image)

## TODO: Test with bootstrapped examples


# Make sure that JPGs work

## Convert images to JPGs

In [ ]:
import io
from PIL import Image

def convert_to_jpg(example):
    example_copy = example.copy()
    for key in ['image_1', 'image_2']:
        if key in example_copy and isinstance(example_copy[key], Image.Image):
            # Convert to RGB mode (in case it's not already)
            img = example[key].convert('RGB')
            
            # Save as JPG in memory
            buffer = io.BytesIO()
            img.save(buffer, format='JPEG')
            buffer.seek(0)
            
            # Load the JPG back as a PIL Image
            example_copy[key] = Image.open(buffer)
    
    return example_copy.with_inputs(*example.inputs().keys())

# Convert all images in the dataset to JPG
examples_jpg = list(map(convert_to_jpg, updated_valset))

# Verify conversion
print("Original image format:", updated_valset[0]['image_1'].format)
print("Converted image format:", examples_jpg[0]['image_1'].format)


In [ ]:
examples_jpg = list(map(convert_to_jpg, updated_valset))
examples_no_image_1_jpg = list(map(lambda x: convert_to_jpg(x), examples_no_image_1))
examples_no_image_2_jpg = list(map(lambda x: convert_to_jpg(x), examples_no_image_2))
examples_no_actual_image_jpg = list(map(lambda x: convert_to_jpg(x), examples_no_actual_image))

mmmu = MMMUModule()
print(examples_no_image_1_jpg[0].inputs())
print(mmmu(**examples_no_image_1_jpg[0].inputs()))
print(examples_no_image_1_jpg[0]["image_1"].format)

In [ ]:
normal = evaluate_mmmu(mmmu, devset=examples_jpg)
no_image_1 = evaluate_mmmu(mmmu, devset=examples_no_image_1_jpg)
no_image_2 = evaluate_mmmu(mmmu, devset=examples_no_image_2_jpg)
no_actual_image = evaluate_mmmu(mmmu, devset=examples_no_actual_image_jpg)
print("Testing on MMMU validation set (N=", len(updated_valset), ")")
print("Score with both images:", normal)
print("Score with image_1 set to black square:", no_image_1)
print("Score with image_2 set to black square:", no_image_2)
print("Score with both images set to black squares:", no_actual_image)

In [ ]:
lm.inspect_history()

# Testing that URLs work

In [ ]:

colors = {
    "White": "FFFFFF",
    "Red": "FF0000",
    "Green": "00FF00",
    "Blue": "0000FF",
    "Yellow": "FFFF00",
    "Cyan": "00FFFF",
    "Magenta": "FF00FF",
    "Gray": "808080",
    "Orange": "FFA500",
    "Purple": "800080"
}
def get_color_image_url(color, file_extension="png"):
    return f"https://placehold.co/300/{colors[color]}/{colors[color]}.{file_extension}"


In [ ]:
import random

def generate_random_2_color_image_examples(n):
    examples = []
    for _ in range(n):
        color_1, color_2 = random.sample(list(colors.keys()), 2)
        chosen_color = color_1 if random.random() < 0.5 else color_2
        chosen_image = "image_1" if chosen_color == color_1 else "image_2"
        example_kwargs = {
            "image_1": get_color_image_url(color_1),
            "image_2": get_color_image_url(color_2),
            "question": f"What color is {chosen_image}?",
            "answer": chosen_color
        }
        examples.append(dspy.Example(**example_kwargs).with_inputs("image_1", "image_2", "question"))
    return examples

examples = generate_random_2_color_image_examples(100)
print(examples[0])


In [ ]:
class ColorSignature(dspy.Signature):
    """Output the color of the designated image."""
    image_1: dspy.Image = dspy.InputField(desc="An image")
    image_2: dspy.Image = dspy.InputField(desc="An image")
    question: str = dspy.InputField(desc="A question about the image")
    answer: str = dspy.OutputField(desc="The color of the designated image")
color_program = dspy.Predict(ColorSignature)


In [ ]:
print(examples[0])
print(color_program(**examples[0].inputs()))

In [ ]:
few_shot_optimizer = dspy.BootstrapFewShot(metric=answer_exact_match, max_bootstrapped_demos=3, max_labeled_demos=10)
smaller_few_shot_optimizer = dspy.BootstrapFewShot(metric=answer_exact_match, max_bootstrapped_demos=1, max_labeled_demos=1)
dataset = generate_random_2_color_image_examples(1000)
trainset = dataset[:200]
validationset = dataset[200:400]
evaluate_colors = Evaluate(metric=answer_exact_match, num_threads=300, devset=validationset)

In [ ]:
compiled_color_program = few_shot_optimizer.compile(color_program, trainset=trainset)
compiled_smaller_color_program = smaller_few_shot_optimizer.compile(color_program, trainset=trainset)
print(evaluate_colors(color_program))
print(evaluate_colors(compiled_color_program))
print(evaluate_colors(compiled_smaller_color_program))

In [ ]:
print(compiled_color_program(**validationset[0].inputs()))
lm.inspect_history()

# TODO(Isaac): Delete; Archive of old experiments

In [ ]:
dataset = DataLoader().from_huggingface("Alanox/stanford-dogs", split="full", input_keys=("image",), trust_remote_code=True)

In [ ]:
# rename the field from "image" to "image_1"
def rename_field(example, old_name, new_name):
    try:
        example[new_name] = example[old_name]
        del example[old_name]
    except Exception:
        pass
    return example
    
dog_dataset = list(map(rename_field, dataset, ["image"]*len(dataset), ["image_1"]*len(dataset)))
dog_dataset2 = list(map(rename_field, dog_dataset, ["target"]*len(dog_dataset), ["answer"]*len(dog_dataset)))
dog_dataset3 = list(map(lambda x: x.with_inputs("image_1"), dog_dataset2))
dog_dataset = dog_dataset3
random.shuffle(dog_dataset)

In [ ]:
class DogPictureSignature(dspy.Signature):
    """Output the dog breed of the dog in the image."""
    image: dspy.Image = dspy.InputField(desc="An image of a dog")
    answer: str = dspy.OutputField(desc="The dog breed of the dog in the image")

class DogPicture(dspy.Module):
    def __init__(self) -> None:
        self.predictor = dspy.ChainOfThought(DogPictureSignature)
    
    def __call__(self, **kwargs):
        return self.predictor(**kwargs)

dog_picture = DogPicture()

example = dspy.Example(image=dspy.Image.from_url("https://i.pinimg.com/564x/78/f9/6d/78f96d0314d39a1b8a849005123e166d.jpg"))
print(dog_picture(**example.inputs()))
# print(dog_picture(**dog_dataset[0].inputs()))

In [ ]:
# TODO: Test inline signature
# TODO: Test json adapter